In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
pd.options.display.max_columns = None

# Importing pickle file

In [2]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [3]:
tracks.head()

,url,track_clean_new,artist_clean_new
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk"
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey"
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane"


In [4]:
tracks.rename(columns={'track_clean_new':'track','artist_clean_new':'artist'}, inplace=True)
tracks.head()

,url,track,artist
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk"
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey"
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane"


# lastfm
In this part of the process I will extract form the LastFM API all the data related to the tracks and artist genres.

First, I need to configure my credentials.

In [12]:
load_dotenv()

True

In [13]:
api = os.getenv("lastfm-id")
username = os.getenv("lastfm-user")

In [16]:
## me voy a hacer una función con los datos de la petición
def getLastFMData(params):
    headers = {'user-agent': username} ## defino mi usuario
    url = 'https://ws.audioscrobbler.com/2.0/' ##url a la que voy a contactar

    ## configuro los params para hacer la petición
    params['api_key'] = api
    params['format'] = 'json'

    res = requests.get(url, headers=headers, params=params)
    return res

In [54]:
res = getLastFMData({
    'method': 'artist.getInfo',
    'artist':'Harry Styles',
})
res.json()['artist']['bio']

{'links': {'link': {'#text': '',
   'rel': 'original',
   'href': 'https://last.fm/music/Harry+Styles/+wiki'}},
 'published': '24 Aug 2011, 17:11',
 'summary': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, toured the world, and sold millions of albums before going solo in 2016. Styles was raised in Holmes Chapel, Cheshire, and he made his first foray into music with a high-school band named White Eskimo. In 2010, he left home and joined the seventh season of The X Factor <a href="https://www.last.fm/music/Harry+Styles">Read more on Last.fm</a>',
 'content': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, to

In [19]:
def getArtistInfo(artist):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'artist.getInfo',
        'artist':  artist,
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    try:
        #tags = [tag['name'] for tag in res.json()['toptags']['tag'][:3]]
        return res.json()['artist']['bio'], res.json()['artist']['tags']['tag']
    except:
        return np.nan

First I try on a slice of my dataframe

In [21]:
tqdm.pandas()

test = tracks.sample(100)
test[['bio', 'artist_tag']] = test.progress_apply(lambda x: getArtistInfo(x.artist), axis=1, result_type='expand')

100%|██████████| 100/100 [00:18<00:00,  5.30it/s]


In [22]:
test.head()

,url,track,artist,bio,artist_tag
5679087,https://open.spotify.com/track/1kK6QRifPUPIWaR...,thank you (feat. big sean),dj khaled,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w..."
5678796,https://open.spotify.com/track/5DiuFfnqqyWKat4...,celebrate (feat. travis scott & post malone),dj khaled,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w..."
7014584,https://open.spotify.com/track/4IO8X9W69dIQe0E...,immortal,21 savage,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'trap', 'url': 'https://www...."
13657314,https://open.spotify.com/track/0n0XAmYCMWlzj3W...,happy together (feat. ray toro),gerard way,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'alternative rock', 'url': '..."
14374123,https://open.spotify.com/track/1AgSUWSzmza5jF6...,thru the night (feat. bryson tiller),jack harlow,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'rap', 'url': 'https://www.l..."


This is for the whole dataframe:

In [24]:
tqdm.pandas()

#test = tracks.sample(100)
tracks[['bio', 'artist_tag']] = tracks.progress_apply(lambda x: getArtistInfo(x.artist), axis=1, result_type='expand')

100%|██████████| 14659/14659 [43:32<00:00,  5.61it/s]  


In [25]:
tracks.head()

,url,track,artist,bio,artist_tag
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w..."
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w..."
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk","{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'mistagged', 'url': 'https:/..."
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []}
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []}


Now I get the tags for each individual track:

In [26]:
res = getLastFMData({
    'method': 'track.getInfo',
    'artist':'Harry Styles',
    'track':'As It Was'
})
res.json()['track']['toptags']#['published']#[0]['name']

{'tag': [{'name': 'pop', 'url': 'https://www.last.fm/tag/pop'},
  {'name': 'rock', 'url': 'https://www.last.fm/tag/rock'},
  {'name': 'indie pop', 'url': 'https://www.last.fm/tag/indie+pop'},
  {'name': 'synthpop', 'url': 'https://www.last.fm/tag/synthpop'},
  {'name': 'british', 'url': 'https://www.last.fm/tag/british'}]}

In [27]:
def getTrackTags(artist, track):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'track.getInfo',
        'artist':  artist,
        'track': track,
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    try:
        #tags = [tag['name'] for tag in res.json()['toptags']['tag'][:3]]
        return res.json()['track']['toptags'], res.json()['track']['wiki']['published'] ## no va muy bien esta parte y solo me extrae tres géneros... 
    except:
        return np.nan, np.nan

This is how I will get info for a slice of my dataframe:

In [31]:
tqdm.pandas()

test = tracks.sample(100)
test[['track_tag', 'published']] = test.progress_apply(lambda x: getTrackTags(x.artist, x.track), axis=1, result_type='expand')

100%|██████████| 100/100 [00:18<00:00,  5.39it/s]


In [32]:
print(test['track_tag'].isnull().sum())
test.head()

63


,url,track,artist,bio,artist_tag,track_tag,published
7151690,https://open.spotify.com/track/4qBejt9FeqLI9bl...,jingle bells (feat. the puppini sisters),michael bublé,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'jazz', 'url': 'https://www....",NaN,NaN
21750268,https://open.spotify.com/track/15vzANxN8G9wWfw...,paris,the chainsmokers,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'electronic', 'url': 'https:...","{'tag': [{'name': 'electronic', 'url': 'https:...","14 Jan 2017, 12:34"
2377815,https://open.spotify.com/track/4UQajPk0EPvbwWT...,moving along,5 seconds of summer,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'pop rock', 'url': 'https://...","{'tag': [{'name': 'love at first listen', 'url...","12 Jan 2022, 22:22"
49027,https://open.spotify.com/track/5GFDrUTLGJix84s...,caroline,aminé,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'rap', 'url': 'https://www.l...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w...","15 Sep 2016, 06:15"
21770864,https://open.spotify.com/track/3ddxe0WYUpNPtSn...,i still have faith in you,abba,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'pop', 'url': 'https://www.l...","{'tag': [{'name': 'pop', 'url': 'https://www.l...","27 Apr 2018, 12:21"


This is for the whole dataset:

In [34]:
tqdm.pandas()

#test = tracks.sample(100)
tracks[['track_tag', 'published']] = tracks.progress_apply(lambda x: getTrackTags(x.artist, x.track), axis=1, result_type='expand')

100%|██████████| 14659/14659 [48:47<00:00,  5.01it/s] 


In [37]:
print(tracks['track_tag'].isnull().sum(), tracks['artist_tag'].isnull().sum())
tracks.head()

9864 86


,url,track,artist,bio,artist_tag,track_tag,published
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w...","{'tag': [{'name': 'trap', 'url': 'https://www....","30 Jan 2018, 04:41"
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w...","{'tag': [{'name': 'trap', 'url': 'https://www....","27 Oct 2016, 02:34"
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk","{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'mistagged', 'url': 'https:/...",NaN,NaN
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN


Cleaning the new bio, artists and track tags

In [46]:
bio = tracks['bio'].apply(pd.Series)
artist_tag = tracks['artist_tag'].apply(pd.Series)
#artist_tag['tag'] = artist_tag['tag'].apply(pd.Series)

In [50]:
artist_tag['tag'].apply(pd.Series)

C:\Users\annas\AppData\Local\Temp\ipykernel_6880\2992344966.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  artist_tag['tag'].apply(pd.Series)


,0,1,2,3,4
8246,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'trap', 'url': 'https://www.last.fm/t...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'atlanta', 'url': 'https://www.last.f..."
8247,"{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'rnb', 'url': 'https://www.last.fm/ta...","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'Canadian', 'url': 'https://www.last...."
8248,"{'name': 'mistagged', 'url': 'https://www.last...","{'name': 'Daft Punk', 'url': 'https://www.last...","{'name': 'mistagged artist', 'url': 'https://w...","{'name': 'The Weeknd', 'url': 'https://www.las...",NaN
8249,NaN,NaN,NaN,NaN,NaN
8250,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
26131455,NaN,NaN,NaN,NaN,NaN
26131463,NaN,NaN,NaN,NaN,NaN
26131464,NaN,NaN,NaN,NaN,NaN
26131472,"{'name': 'rap', 'url': 'https://www.last.fm/ta...","{'name': 'Hip-Hop', 'url': 'https://www.last.f...","{'name': 'powerviolence', 'url': 'https://www....","{'name': 'hip hop', 'url': 'https://www.last.f...","{'name': 'psytrance', 'url': 'https://www.last..."


In [47]:
pd.concat([tracks, bio, artist_tag], axis = 1)

,url,track,artist,bio,artist_tag,track_tag,published,links,published,summary,content,0,tag,0
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w...","{'tag': [{'name': 'trap', 'url': 'https://www....","30 Jan 2018, 04:41","{'link': {'#text': '', 'rel': 'original', 'hre...","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",NaN,"[{'name': 'Hip-Hop', 'url': 'https://www.last....",NaN
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Hip-Hop', 'url': 'https://w...","{'tag': [{'name': 'trap', 'url': 'https://www....","27 Oct 2016, 02:34","{'link': {'#text': '', 'rel': 'original', 'hre...","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",NaN,"[{'name': 'Hip-Hop', 'url': 'https://www.last....",NaN
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk","{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'mistagged', 'url': 'https:/...",NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Weeknd...",,NaN,"[{'name': 'mistagged', 'url': 'https://www.las...",NaN
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/The+Chains...",,NaN,[],NaN
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Rae+Sremmu...",,NaN,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26131455,https://open.spotify.com/track/6jDOs0wGlRn4jCg...,butterflies,fase yoda,"{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Fase+Yoda""...",,NaN,[],NaN
26131463,https://open.spotify.com/track/7eQHxigpuDJjCG5...,butterflies,"max, ali gatie","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/MAX,+Ali+G...",,NaN,[],NaN
26131464,https://open.spotify.com/track/0kgRRHa5CMeCciI...,colt 45 (country remix),"cooper alan, rvshvd","{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []},NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Cooper+Ala...",,NaN,[],NaN
26131472,https://open.spotify.com/track/7GLqR9ToJLb0PV3...,mmm mmm (feat. atl jacob),kali,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'rap', 'url': 'https://www.l...",NaN,NaN,"{'link': {'#text': '', 'rel': 'original', 'hre...","10 Aug 2007, 15:12",There are 2 artists currently using the Kali n...,There are 2 artists currently using the Kali n...,NaN,"[{'name': 'rap', 'url': 'https://www.last.fm/t...",NaN


# Export

In [38]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'wb') as data_lastfm:
    pickle.dump(tracks, data_lastfm)